# TGDS Hybrid Model

In [ ]:
import context
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import src.data.data_preprocessing as util
import src.models.lstm_model as lstm

tf.compat.v1.set_random_seed(1)

### Set Hyperparameters

In [ ]:
HYPER_PARAMS = {
    'n_epochs': 15,                   # number of training epochs
    'd_sample': 4,                    # subsampling ratio
    'gauss_sigma': 10,                # smoothing factor
    'd_t': 0.25,                      # charge integration factor
    'feature_range_cur_low': -1,      # lower bound of current input feature after scaling
    'feature_range_cur_high': 1,      # upper -||-
    'feature_range_charge_low': -1,   # lower bound of charge input feature after scaling
    'feature_range_charge_high': 1,   # upper -||-
    'feature_range_volt_low': -1,     # lower bound of voltage label after scaling
    'feature_range_volt_high': 1,     # upper -||-
    'boundary_cur_low': -10,          # lower bound of current value range in A
    'boundary_cur_high': 10,          # upper -||-
    'boundary_charge_low': -33.2,     # lower bound of accumulated current value range 
    'boundary_charge_high': 33.2,     # upper -||-
    'boundary_voltage_low': 3.304,    # lower bound of voltage value range in V
    'boundary_voltage_high': 3.982,   # upper -||-
    'n_steps': 100,                   # defines M in the M-to-1 LSTM structure
    'n_features': 3,                  # number of input features
    'n_lstm_units_1': 50,             # number of LSTM units in the first layer
    'alpha_1': 0.1,                   # alpha value for LeakyReLU 
    'n_lstm_units_2': 20,             
    'alpha_2': 0.1,                   
    'n_dense_units': 10,              # number of dense units in the output layer
    'activation_output_layer': 'tanh',# output activation function
    'dropout': 0.2,                   # dropout factor
    'learning_rate': 0.001,           # learning rate
    'optimizer': 'Adam',              # optimizer
    'metric': 'mae',                  # performance metric during training 
    'loss_funcs': ['mse'],            # loss functions
    'lambda_mse': 1,                  # loss function weighting factor
    'lambda_apx': 0.1,
    'lambda_mon': 0.1,
    'theory_model': 5595,
}

### Prepare Training/Validation/Test Data

In [ ]:
# train_data = ['Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A']
train_data = [
    'Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A',
    'Profile 10A Run 040618', 'Profile 10A Run 040618', 'Profile 10A Run 040618', 'Profile 10A Run 040618', 'Profile 10A Run 040618','Profile 10A Run 040618', 
    'Profile 10A Run 080618', 'Profile 10A Run 080618', 'Profile 10A Run 080618', 'Profile 10A Run 080618', 'Profile 10A Run 080618', 'Profile 10A Run 080618',
    'Profile 10A Run 070618_3', 'Profile 10A Run 070618_3', 'Profile 10A Run 070618_3', 'Profile 10A Run 070618_3', 'Profile 10A Run 070618_3', 'Profile 10A Run 070618_3',
    'Profile 10A Run 070618_2', 'Profile 10A Run 070618_2', 'Profile 10A Run 070618_2', 'Profile 10A Run 070618_2', 'Profile 10A Run 070618_2','Profile 10A Run 070618_2',
    'Profile 10A Run 070618', 'Profile 10A Run 070618', 'Profile 10A Run 070618', 'Profile 10A Run 070618', 'Profile 10A Run 070618','Profile 10A Run 070618','Profile 10A Run 070618'
]

# select arbitrary profile for validation
validation_profile = np.random.choice(train_data, 1) 

# select arbitrary profile for testing
test_data = ['Profile 25A Run 2', 'Profile 25A Run 040618', 'Profile 25A Run 070618', 'Profile 25A Run 070618_3']
test_profile = np.random.choice(test_data, 1) 

# prepare input data
X_train, y_train, scalers_train = util.prepare_hybrid_input(HYPER_PARAMS, train_data, 0, 4)
X_validation, y_validation, _ = util.prepare_hybrid_input(HYPER_PARAMS, validation_profile, 0, 4)
X_test, y_test, _ = util.prepare_hybrid_input(HYPER_PARAMS, test_profile, 0, 4)

### Initialize and Train Model

In [ ]:
lstm = lstm.Model()
lstm.initialize(HYPER_PARAMS)

In [ ]:
_, fig = lstm.train(X_train, y_train, scalers_train)

# save model, hyperparameters and plots
MODEL_ID = str(np.random.randint(10000))

lstm.model.save('../../../models/TGDS/' + str(MODEL_ID))
np.save('../../../models/TGDS/' + str(MODEL_ID) + '/hyperparameters', HYPER_PARAMS)
fig.savefig('../../../reports/figures/theory_guided_hybrid-' + str(MODEL_ID) + '-learning_curve.png')

### Test Model

In [ ]:
yhat_train_unscaled, _, _, _, _, fig = lstm.test(X_train, y_train, X_validation, y_validation, X_test, y_test, scalers_train)
print('Validation:', validation_profile)
print('Test:', test_profile)

# save plots and predicted sequences
np.save('../../../models/TGDS/' + str(MODEL_ID) + '/predictions', yhat_train_unscaled)
fig.savefig('../../../reports/figures/theory_guided_hybrid-' + str(MODEL_ID) + '-validation&test_profiles.png')

### Prepare Data for Use Cases

In [ ]:
test_profiles_usecase_1 = np.random.choice(train_data, 1) 
test_profiles_usecase_2 = ['Profile 10A Run 040618' ]
test_profiles_usecase_3 = ['Profile 10A 3x']

X_case_1, y_case_1, _ = util.prepare_hybrid_input(HYPER_PARAMS, test_profiles_usecase_1, 0, 4)
X_case_2, y_case_2, _ = util.prepare_hybrid_input(HYPER_PARAMS, test_profiles_usecase_2, 0, 4)
X_case_3, y_case_3, _ = util.prepare_hybrid_input(HYPER_PARAMS, test_profiles_usecase_3, 0, 4)

### Test Model on Use Cases

In [ ]:
print('Use Case 1:', test_profiles_usecase_1)
print('Use Case 2:', test_profiles_usecase_2)
print('Use Case 3:', test_profiles_usecase_3)

lstm.test_usecases(X_train, y_train, X_case_1, y_case_1, X_case_2, y_case_2, X_case_3, y_case_3, scalers_train)